In [2]:
# TASK-9A: Please apply the following feature engineering steps on the numeric features of the AD dataset.
#
# Divide by NACCBRNV (MRI)
# Multiply by NACCAGEB
#
# TASK-9B: Please apply Gaussian Naive Bayes algorithm on the numeric values (after the transformation) to predict the CDRGLOB values (class-based). Train-test split, train by grid search (k=5) and optimize for the parameter values.
#
# TASK-9C: Please apply a one-hot encoding method on the categorical data of the AD dataset.
#
# TASK–9D: Please apply Multinomial Naive Bayes algorithms on the one-hot encoded categorical data of the AD dataset and predict the labels (CDRGLOB).
#
# TASK-9E: Please merge the normalized numeric values and one-hot encoded categorical values and then apply the following regression methods (Linear, Ridg and Lasso) to predict the labels as numeric values. Please do not forget to optimize them and use R2 for scoring.
#
# TASK-9F: Please find and apply the classification versions of the following algorithms (Linear, Ridge and Lasso) with the hyperparameter optimization.

In [15]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from keras.losses import mean_squared_error

In [4]:
df = pd.read_excel("Temporary_data3_Left_Right_Copy.xlsx")
df

,SEX,EDUC,MARISTAT,INDEPEND,RESIDENC,NACCFAM,ANYMEDS,SMOKYRS,NACCTBI,DIABETES,...,RSUPFR,RSUPFRM,RSUPPAR,RSUPPARM,RSUPTEM,RSUPTEMM,RSUPMAR,RSUPMARM,RTRTEM,RTRTEMM
0,2,18,1,1,1,1,1,0,0,0,...,27.9038,2.2157,10.3481,1.7014,13.4250,1.7419,9.8700,1.6792,0.8906,1.1947
1,1,11,1,1,1,1,1,0,0,0,...,29.0600,2.6900,9.2500,1.6300,14.8800,2.3000,7.2700,1.8900,1.3800,2.0300
2,2,16,1,1,1,1,1,10,0,1,...,33.0000,2.5400,13.6900,1.5600,16.2300,2.0700,10.1100,1.8400,1.0100,1.5000
3,1,14,1,1,1,0,1,0,0,1,...,31.6800,2.4100,9.6400,1.4500,15.1100,1.6400,10.3900,1.8300,0.7400,1.1000
4,1,16,1,1,2,0,1,50,0,0,...,28.0631,1.9501,9.1125,1.0427,13.3763,1.6270,7.5844,1.4595,0.6638,0.8527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,1,20,1,1,1,1,1,0,0,0,...,24.6892,2.6528,10.5460,2.1712,14.1128,2.8833,9.3176,2.3582,0.6735,1.0270
1350,1,16,1,2,1,0,1,0,0,0,...,24.7908,2.0993,10.4094,1.4855,10.7880,1.9027,10.2034,2.0484,0.7267,0.7946
1351,2,14,2,1,1,1,1,0,0,0,...,23.8500,2.1200,12.3900,1.5300,12.6600,1.9900,9.4800,1.7400,1.1800,1.5900
1352,2,20,3,1,1,1,1,25,0,0,...,31.3500,2.4300,10.8200,1.7000,13.7700,2.1000,11.1200,2.1400,1.3100,1.5200


TASK 9-A

In [5]:
newdf = df.select_dtypes(include=['float64']).columns
df[newdf]

,NACCBMI,NACCICV,NACCBRNV,NACCWMVL,CSFVOL,GRAYVOL,WHITEVOL,WMHVOL,CEREALL,CERETISS,...,RSUPFR,RSUPFRM,RSUPPAR,RSUPPARM,RSUPTEM,RSUPTEMM,RSUPMAR,RSUPMARM,RTRTEM,RTRTEMM
0,25.1,1399.056,1118.929,481.802,279.718,637.536,481.393,0.4088,1189.23,952.446,...,27.9038,2.2157,10.3481,1.7014,13.4250,1.7419,9.8700,1.6792,0.8906,1.1947
1,30.3,1578.180,1107.680,477.910,468.740,631.530,476.150,1.7600,1382.83,975.620,...,29.0600,2.6900,9.2500,1.6300,14.8800,2.3000,7.2700,1.8900,1.3800,2.0300
2,23.2,1455.550,1187.760,527.470,266.410,661.670,526.090,1.3800,1247.03,1035.400,...,33.0000,2.5400,13.6900,1.5600,16.2300,2.0700,10.1100,1.8400,1.0100,1.5000
3,30.4,1494.650,1115.330,478.960,376.840,638.850,476.480,2.4800,1248.03,965.820,...,31.6800,2.4100,9.6400,1.4500,15.1100,1.6400,10.3900,1.8300,0.7400,1.1000
4,20.8,1494.067,1014.743,444.578,471.128,578.361,436.382,8.1956,1281.46,885.615,...,28.0631,1.9501,9.1125,1.0427,13.3763,1.6270,7.5844,1.4595,0.6638,0.8527
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,27.8,1472.904,1141.103,519.282,329.904,623.718,517.385,1.8967,1255.75,989.959,...,24.6892,2.6528,10.5460,2.1712,14.1128,2.8833,9.3176,2.3582,0.6735,1.0270
1350,28.3,1723.724,1192.320,535.784,521.124,666.816,525.504,10.2797,1478.48,1026.640,...,24.7908,2.0993,10.4094,1.4855,10.7880,1.9027,10.2034,2.0484,0.7267,0.7946
1351,25.8,1450.360,1068.170,449.420,373.560,627.380,440.790,8.6300,1232.13,918.530,...,23.8500,2.1200,12.3900,1.5300,12.6600,1.9900,9.4800,1.7400,1.1800,1.5900
1352,44.2,1528.400,1105.820,487.800,391.970,648.630,457.190,30.6100,1327.44,995.230,...,31.3500,2.4300,10.8200,1.7000,13.7700,2.1000,11.1200,2.1400,1.3100,1.5200


In [6]:
df[newdf] = df[newdf].div(df['NACCBRNV'], axis=0)

df[newdf] = df[newdf].mul(df["NACCAGEB"], axis=0)

In [7]:
df.drop(['NACCAGEB'], axis=1, inplace=True)

In [8]:
y= df["CDRGLOB"]
X= df[newdf]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

TASK 9-B

In [9]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

y_pred = gnb.predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 271 points : 134


In [10]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
accuracy_score(y_test, y_pred)

0.5055350553505535

TASK 9-C

In [11]:
smoothing = np.logspace(0,-9, num=100)
param_grid = dict(var_smoothing=smoothing)
grid = GridSearchCV(gnb, param_grid, cv=10, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

Best cross-validation score: 0.50
Best parameters:  {'var_smoothing': 1.0}
Test set score: 0.53


In [12]:
gnb = grid.best_estimator_
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
accuracy_score(y_test, y_pred)

0.5276752767527675

TASK 9-D

In [13]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)
accuracy_score(y_test, y_pred)

alpha = [0.01, 0.1, 1, 10, 100]
param_grid = dict(alpha=alpha)
grid = GridSearchCV(mnb, param_grid, cv=10, scoring='accuracy')
grid.fit(X_train, y_train)

print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Best parameters: ", grid.best_params_)
print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))

Best cross-validation score: 0.54
Best parameters:  {'alpha': 10}
Test set score: 0.59


TASK 9-E

In [17]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("R2 score: {:.2f}".format(r2_score(y_test, y_pred)))
print("MSE: {:.2f}".format(mean_squared_error(y_test, y_pred)))

#Ridge Regression

from sklearn.linear_model import Ridge

ridge = Ridge()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print("R2 score: {:.2f}".format(r2_score(y_test, y_pred)))
print("MSE: {:.2f}".format(mean_squared_error(y_test, y_pred)))

#Lasso Regression

from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print("R2 score: {:.2f}".format(r2_score(y_test, y_pred)))
print("MSE: {:.2f}".format(mean_squared_error(y_test, y_pred)))

R2 score: 0.34
MSE: 0.40
R2 score: 0.46
MSE: 0.33
R2 score: 0.06
MSE: 0.57


TASK 9-F

In [23]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

lr = RidgeClassifier()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Accuracy score: {:.2f}".format(accuracy_score(y_test, y_pred)))

#Ridge Classification

from sklearn.linear_model import RidgeClassifier

ridge = RidgeClassifier()
ridge.fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print("Accuracy score: {:.2f}".format(accuracy_score(y_test, y_pred)))

#Lasso Classification

from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(X_train, y_train)
y_pred = lasso.predict(X_test)
print("Accuracy score: {:.2f}".format(accuracy_score(y_test, y_pred)))

Accuracy score: 0.65
Accuracy score: 0.65


ValueError: Classification metrics can't handle a mix of multiclass and continuous targets